# Output Parsers

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

- `get_format_instructions() -> str`: A method which returns a string containing instructions for how the output of a language model should be formatted.
- `parse(str) -> Any`: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

Below we go over some examples of output parsers.

## Structured Output Parser

This output parser can be used when you want to return multiple fields.

In [4]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [6]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

Here we define the response schema we want to receive.

In [5]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

In [6]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

We can now use this to format a prompt to send to the language model, and then parse the returned result.

In [7]:
model = OpenAI(temperature=0)

In [10]:
_input = prompt.format(question="what's the capital of paris")
output = model(_input)

In [11]:
output_parser.parse(output)

{'answer': 'The capital of Paris is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}

## CommaSeparatedListOutputParser

This output parser can be used to get a list of items as output.

In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [2]:
output_parser = CommaSeparatedListOutputParser()

In [8]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
model = OpenAI(temperature=0)

In [10]:
_input = prompt.format(subject="ice cream flavors")
output = model(_input)

In [11]:
output_parser.parse(output)

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']